In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
import os
import pandas as pd

def run_etl():

    raw = pd.read_csv('/opt/airflow/dags/data/raw/deliveries_sample.csv')
    raw['delivery_time_days'] = (pd.to_datetime(raw['delivered_at']) - pd.to_datetime(raw['order_date'])).dt.days
    processed = raw.dropna()
    processed.to_parquet('/opt/airflow/dags/data/processed/cleaned_deliveries.parquet')
    print("ETL concluído!")

default_args = {
    'owner': 'sidney',
    'start_date': datetime(2025, 1, 1),
    'retries': 1,
}

with DAG(
    'etl_logistica_dag',
    default_args=default_args,
    description='Pipeline de ETL de logística',
    schedule_interval='0 6 * * *',  # roda todos os dias às 6h
    catchup=False,
) as dag:

    etl_task = PythonOperator(
        task_id='executar_etl',
        python_callable=run_etl,
    )

etl_task
